In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_C_049, input_features_C_049 = pre_models.Input_C_049('weights/')
    
    outputs = ['Input_C_049']

    preprocessings = ['min_max_normalization']

    models = [model_C_049]

    input_features = [input_features_C_049]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=8
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_049.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_047', 'Input_C_048', 'Input_C_137', 'Input_C_044',
       'Input_C_041', 'Input_C_136', 'Input_C_043', 'Input_A4_011'],
      dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.006445387908787211
[0.0049702157294444485, 0.014162584831045644, 0.002936556938264458, 0.019685776422847537, 0.003150611296923237, 0.01113212695466879, 0.0058610306080245575, 0.0002610205411221014, 0.0002067632793037513, 0.002087192486227574]


test_r2_ave: -856.7262853575033
[-297.93044523980836, -2002.011389606389, -155.96531820078272, -4000.7120249776353, -120.20892460224115, -1500.9687307074098, -435.24525865997214, -0.31018310311098984, 0.36844431390266863, -54.2790227915864]


## Linear Regression Baseline

In [9]:
lr_rmse_ave_fea(skew_data,8)

train_rmse: 0.00022172789652128146
test_rmse: 0.00023546602040798217
test_r2: 0.3190337509979494


## Lasso

In [10]:
laso_rmse_ave(skew_data[title],y,7e-06)

test_rmse_ave: 0.00022505173779003708
[0.00023712937718824043, 0.00024464335206266505, 0.00019398886669105128, 0.0002498756010646379, 0.0002084259236082467, 0.0002213630122603428, 0.00024285143796758351, 0.0002134882864157711, 0.000204171679208436, 0.000234579841433396]


test_r2_ave: 0.3328288650927885
[0.31955912971793454, 0.402324838111494, 0.3150159413336382, 0.3552543226233065, 0.46954516087787335, 0.406098117201805, 0.25103055637760296, 0.12354277483104381, 0.38417710957315665, 0.30174070028002953]


## ElasticNet

In [11]:
ElasticNet_rmse_ave(skew_data[title],y,[0.005],0.01)

test_rmse_ave: 0.0002334562317344059
[0.0002474261109276999, 0.0002676054513932509, 0.00019495238154806539, 0.0002684603591224828, 0.00022497666705178292, 0.00023123371853646016, 0.00024082689531972214, 0.00020479139538548885, 0.00021187269274605564, 0.0002424166453130505]


test_r2_ave: 0.289004584499655
[0.25918336153952937, 0.284864589884112, 0.3081946080047384, 0.25578045893280477, 0.3819552759243223, 0.35195238492842806, 0.26346614377713895, 0.19349691459101115, 0.33684538106087103, 0.25430672635359397]


## SVR

In [12]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,8,model)

train_rmse: 0.0002465000900666068
test_rmse: 0.000256249934256984
test_r2: 0.18580959310929768


## XGB

In [13]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 180, 
              max_depth = 2, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.6,
              reg_alpha =  0,
              reg_lambda = 0.1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.00023097241472560143
[0.00023497692839814167, 0.0002612631985521791, 0.00018792810101619332, 0.0002620130646339197, 0.0002195300099393976, 0.00024036067581113043, 0.00024365881063029616, 0.0002064596391657435, 0.00020436786283632575, 0.0002491658562726869]


test_r2_ave: 0.30313117840520115
[0.331855935077917, 0.3183603429246522, 0.35714903023190947, 0.29109730188911265, 0.4115186004568713, 0.2997849789673104, 0.24604229990561377, 0.1803037451337679, 0.38299308235594, 0.21220646710891722]


## RandomForest

In [14]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 9,
            min_samples_split = 2,
            max_leaf_nodes = 10,
            min_samples_leaf =5,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.0002320167707026086
[0.0002289979038397525, 0.0002522624481159855, 0.00019535913352243056, 0.00025549552284764664, 0.00020496587873325355, 0.0002361910154974464, 0.00025281709670211335, 0.0002283193813180385, 0.0002120616160595685, 0.00025369771038985024]


test_r2_ave: 0.2876841704003815
[0.36542540091539866, 0.36451751880399363, 0.3053048073402945, 0.3259263913345103, 0.4870109630952959, 0.32386823936672027, 0.1882998608790586, -0.002462604389031009, 0.33566220616670595, 0.1832889204908682]


In [15]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=7e-06)
elastic_mod=ElasticNet(alpha=[0.005], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 180, 
              max_depth = 2, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.6,
              reg_alpha =  0,
              reg_lambda = 0.1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 9,
            min_samples_split = 2,
            max_leaf_nodes = 10,
            min_samples_leaf =5,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.00022379306308581583
[0.00023113348209521405, 0.00024389307423465105, 0.00018616056774863678, 0.0002592490725168511, 0.00021056825777861205, 0.0002208807701764664, 0.00024303047892879775, 0.00020625020799163338, 0.00020237744701812055, 0.000234387272369175]


test_r2_ave: 0.3431677956708818
[0.35353443817881103, 0.4059851444510997, 0.3691846650742481, 0.3059749305911702, 0.4585844138839622, 0.4086829439150784, 0.24992580162633826, 0.18196588965152827, 0.3949530827722936, 0.3028866465642882]


In [16]:

stack_mod = StackingRegressor(regressors=[vote_mod, lr, elastic_mod, random_mod], 
                           meta_regressor=lasso_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.00022505173779003708
[0.00023712937718824043, 0.00024464335206266505, 0.00019398886669105128, 0.0002498756010646379, 0.0002084259236082467, 0.0002213630122603428, 0.00024285143796758351, 0.0002134882864157711, 0.000204171679208436, 0.000234579841433396]


test_r2_ave: 0.3328288650927885
[0.31955912971793454, 0.402324838111494, 0.3150159413336382, 0.3552543226233065, 0.46954516087787335, 0.406098117201805, 0.25103055637760296, 0.12354277483104381, 0.38417710957315665, 0.30174070028002953]


In [17]:
ave(skew_data[title],y,0.15,stack_mod,0.7,vote_mod,0.15,lasso_mod)

test_rmse_ave: 0.00022363495007356036
[0.0002325207216233736, 0.0002438343348851036, 0.0001878834481493141, 0.000256100945788478, 0.0002095221543904533, 0.00022007089952867624, 0.00024251406278557886, 0.00020764995157300793, 0.00020250942059391855, 0.00023374356141769954]


test_r2_ave: 0.34339756492733287
[0.34575111028102057, 0.4062712357608723, 0.35745448458772655, 0.3227280327959531, 0.4639505579072454, 0.4130111829820875, 0.25311008439908533, 0.1708248248220151, 0.39416370389467537, 0.3067104318426479]


In [18]:
name='Input_C_049'
test_pred=[]
seed=[2,4,7,8,3,6,5]
stack_w=0.15
vote_w=0.7
model_w=0.15
model=lasso_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
